<div> A neat way to put D3.js into the iPython/Jupyter notebook</div>

In [11]:
%%writefile f1.template
<!DOCTYPE html>
<html>
    <meta http-equiv="Content-Type" content="text/html;charset=utf-8"/>
    <script type="text/javascript" src="http://mbostock.github.io/d3/talk/20111018/d3/d3.geom.js"></script>
    <script type="text/javascript" src="http://mbostock.github.io/d3/talk/20111018/d3/d3.layout.js"></script>
    <style type="text/css">
    </style>
  
<body>
<!-- nice simple setup template for D3.js -->
    <div id="body">
    <script src="http://d3js.org/d3.v3.min.js"></script>
        <style>
            .node {
                stroke: #fff;
                stroke-width: 1.5px;
            }

            .link {
                stroke: #999;
                stroke-opacity: .6;
            }

        </style>
        <script>
            //init d3
            var width = 960,
                height = 500;
            var color = d3.scale.category20();
            var force = d3.layout.force()
            .charge(-120)
            .linkDistance(30)
            .size([width, height])
            .on("tick", tick);
            var svg = d3.select("body").append("canvas")
            .attr("width", width)
            .attr("height", height);
            var context = svg.node().getContext("2d");
            var nodes = [];
            var links = [];
            var node = svg.selectAll(".node")
            var link = svg.selectAll(".link")
            readFromFile();
        
            //read from JSON
            function readFromFile(){
                var linkToJSON = "https://gist.githubusercontent.com/maheshmurag/1d2c683d171765ba7771/raw/b8a28e7bd07377773cb86894f67fe68173cf9c0f/jsonstr.json";
                d3.json(linkToJSON,
                    function(error, graph) {
                        console.log("read")
                        if (error) throw error;
                        for(var i = 0; i < graph.nodes.length; i++)
                            nodes.push(graph.nodes[i])
                        for(var i = 0; i < graph.links.length; i++)
                            links.push(graph.links[i])
                        start();
                        update();
                    }
                );
            }
            
            function tick(){
                context.clearRect(0, 0, width, height);

                link.attr("x1", function(d) { return d.source.x; })
                    .attr("y1", function(d) { return d.source.y; })
                    .attr("x2", function(d) { return d.target.x; })
                    .attr("y2", function(d) { return d.target.y; });

                context.strokeStyle = "#ccc";
                context.beginPath();
                links.forEach(function(d) {
                  context.moveTo(d.source.x, d.source.y);
                  context.lineTo(d.target.x, d.target.y);
                });
                context.stroke();

                // draw nodes
                context.fillStyle = "steelblue";
                context.beginPath();
                nodes.forEach(function(d) {
                  context.moveTo(d.x, d.y);
                  context.arc(d.x, d.y, 4.5, 0, 2 * Math.PI);
                });
                context.fill();
            }
            function start(){
                console.log("called start method")
                force
                    .nodes(nodes)
                    .links(links)
                    .start();
            }
            function update(){
                link = link
                    .data(links);

                link
                    .enter().append("line")
                    .attr("class", "link")
                    .style("stroke-width", function(d) { return Math.sqrt(d.value); });

                node = node
                    .data(nodes);

                node
                    .enter().append("circle")
                    .attr("class", "node")
                    .attr("r", 5)
                    .style("fill", function(d) { return color(d.group); })
                    .call(force.drag);

                node
                    .append("title")
                    .text(function(d) { return d.name; });

                node.exit().remove();
                link.exit().remove();
            }
        </script>
  </body>
</html>

Overwriting f1.template


In [14]:
from IPython.display import IFrame
import re

def replace_all(txt,d):
    rep = dict((re.escape('{'+k+'}'), str(v)) for k, v in d.items())
    pattern = re.compile("|".join(rep.keys()))
    return pattern.sub(lambda m: rep[re.escape(m.group(0))], txt)    

count=0
def serve_html(s,w,h):
    import os
    global count
    count+=1
    fn= '__tmp'+str(os.getpid())+'_'+str(count)+'.html'
    with open(fn,'w') as f:
        f.write(s)
    return IFrame('files/'+fn,w,h)

def display(w=1000,h=500):
    d={
       'width'      :w,
       'height'     :h,
       }
    with open('f1.template','r') as f:
        s=f.read()
        s= replace_all(s,d)        
    return serve_html(s,w,h)

In [15]:
#ta da! Well you can't view it from the github repo, but clone and give it a go!
display()